# Data Mining:  Unsupervised Exploratory Learning

## $k$-D Tree for Classification and Regression

[$k$-d trees](https://en.wikipedia.org/wiki/K-d_tree) partition $k$-dimensional space in order to organize points and data access.  They are particularly apt for multidimensional key searches.

![](https://upload.wikimedia.org/wikipedia/commons/thumb/b/bf/Kdtree_2d.svg/740px-Kdtree_2d.svg.png)

![](https://upload.wikimedia.org/wikipedia/commons/thumb/2/25/Tree_0001.svg/740px-Tree_0001.svg.png)

There's a sense in which that's a terrible definition, particularly if you're thinking of _physical_ dimensions.  Who needs a 150-dimensional problem?  What we mean by "dimension," though, is "thing that can differ" across our dataset:  factors.  Thus you can build the tree for a customer relations database along location, name, company size, frequency of purchase, etc.

Why use $k$-d trees?  They are very fast at identifying neighboring points by eliminating large swaths of search space.  Because you know that adjacent points are on adjacent branches, it's relatively efficient to classify points together on the basis of the tree structure.

Today we will use a $k$-d tree for identifying similarity of values based on their parameterized description.

In [ ]:
import pandas as pd
from scipy import spatial
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.figsize'] = (10,4)

### Example:  Online News Data

The University of California—Irvine provides [a collection of standard data sets](http://archive.ics.uci.edu/ml/datasets/Online+News+Popularity) suitable for machine learning and data mining applications.  We will use a data set containing a summary of articles published by [Mashable](https://mashable.com/).  The data file contains 39,644 rows and 61 columns (corresponding to 58 predictive attributes, 2 non-predictive attributes and 1 numerical label about article popularity).  The source website contains more information (about dates, content, etc.).

We will use Pandas to represent the data, and SciPy to provide the $k$-d tree data structure and processing.

-   How can you check what the column headers are labeled as?  What about the basic range and order-of-magnitude of the entries?

We will transform the original (numerical) labels to binary labels. Each article will be designated as either popular (label is more than median value, indicated by `True`), or not popular (label is less than median value, indicated by `False`).

#### Fitting a classification tree from an original data set

To construct a $k$-d tree using SciPy:

There's not a really great tool in Python for visualizing this tree (although generally one may be able to use `pygraphviz`).  A highly-dimensional $k$-d tree doesn't really have a good 2D or 3D representation, though.

![](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b6/3dtree.png/500px-3dtree.png)

You can tune the number of leaves from each node using a second parameter in `spatial.KDTree`, although quantifying the relative efficiency is a drawn-out process.  From the [SciPy documentation](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.spatial.KDTree.html):

> High-dimensional nearest-neighbor queries are a substantial open problem in computer science.

We can construct an array of trees and compare their relative performance for lookups.  First, we will sample a subset of the overall data set for efficiency in timing (since this can be a time-intensive process).


In [ ]:
n = 1000
pop_sample = pop.sample( n=n )

leaves = [ 1,2,4,10,20,50,100 ]
trees = []
for leaf in leaves:
    itree = spatial.KDTree( pop_sample.loc[ :,' timedelta':' shares' ],leafsize=leaf )
    trees.append( itree )

for i,tree in enumerate( trees ):
    %timeit tree.query_pairs( 2.0 )

-   Do other operations show a similar or opposite trend?

$k$-d trees can become unbalanced.  They tend to be more effective tools when you don't know the underlying data distribution.


#### Fitting a classification tree from PCA

Right now the dataset has 53 dimensions.  With PCA, we can reduce this to 27 while maintaining 80% of our accuracy:

In [ ]:
covMatrix = pop.loc[ :,' timedelta':' shares' ].cov()

from numpy.linalg import eig
eigV = eig( covMatrix )
eigVal = eigV[ 0 ]
eigVec = eigV[ 1 ]

info = 0.80
sortedValue = sorted( eigVal,reverse=True )
sortedIndex = sorted( range( len( eigVal ) ), key=lambda k:eigVal[ k ],reverse=True)

fig,ax = plt.subplots()
ax.plot( range( 1,len( sortedValue )+1 ),np.cumsum( sortedValue ) / np.sum( sortedValue ),'mo--',linewidth=2 )
ax.plot( range( 1,len( sortedValue )+1),np.ones( len( sortedValue ) )*info,'b-' )
plt.xlabel('Number of principal components')
plt.ylabel('Percentage of information (%)')
plt.show()

Filter out the indices corresponding to the eigenvalues required to hit this information carrying capacity:

In [ ]:
eigIDs = []
for i in range( 0,len( sortedValue ) ):
    if sum( sortedValue[ 0:i+1 ] ) / sum( sortedValue ) > info:
        eigIDs = sortedIndex[ 0:i+1 ]
        break

eigVal_pca = eigVal[ eigIDs ]
eigVec_pca = eigVec[ :,eigIDs ]

Use the PCA eigenvectors to produce the reduced-dimensional data set:

In [ ]:
data_array = pop.loc[ :,' timedelta':' shares' ].as_matrix()
data_pca = data_array.dot( eigVec_pca )
plt.plot( data_pca[ :,0 ],data_pca[ :,1 ],'bo' )
plt.show()

Finally, construct the $k$-d tree from the PCA data:

In [ ]:
pca_tree = spatial.KDTree( data_pca )

To quantify how much "better" this tree is, we can carry out time trials:

In [ ]:
%timeit pca_tree.query_ball_point( np.ones((1,27))*True,5 )
%timeit tree.query_ball_point( np.ones((1,27))*True,5 )

or figure out the relative error rate.

-   How could we measure the error rate?

### Reference

-   K. Fernandes, P. Vinagre and P. Cortez.  (2015)  A Proactive Intelligent Decision Support System for Predicting the Popularity of Online News. *Proceedings of the 17th EPIA 2015 (Portuguese Conference on Artificial Intelligence)*.  Coimbra, Portugal.

## Contributors

These lessons were developed by Erhu Du, Jane Lee, and Neal Davis for Computational Science and Engineering at the University of Illinois.  Development was supported by a grant from MathWorks, Inc.